In [3]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize
import folium 

In [4]:
# I had to use some credential found on the internet because Foursquare registration is not working properly
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ'
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' 
VERSION = '20180604'

In [34]:
LIMIT = 100
cities = ["Roma, RM", 'Milano, MI', 'Firenze, FI']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d131941735") # Category ID for spiritual places
    results[city] = requests.get(url).json()

In [35]:
results["Roma, RM"]

{'meta': {'code': 200, 'requestId': '5fd724c3b8aa6d5abd982759'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': 'With specials', 'key': 'specials'}]},
  'geocode': {'what': '',
   'where': 'roma rm',
   'center': {'lat': 41.89193, 'lng': 12.51133},
   'displayString': 'Rome, Latium, Italy',
   'cc': 'IT',
   'geometry': {'bounds': {'ne': {'lat': 41.98942097197372,
      'lng': 12.615249866611695},
     'sw': {'lat': 41.79461305370221, 'lng': 12.379309999999998}}},
   'slug': 'roma',
   'longId': '72057594041097006'},
  'headerLocation': 'Rome',
  'headerFullLocation': 'Rome',
  'headerLocationGranularity': 'city',
  'query': 'spiritual',
  'totalResults': 125,
  'suggestedBounds': {'ne': {'lat': 41.956038182754874,
    'lng': 12.572689767025722},
   'sw': {'lat': 41.76988005011028, 'lng': 12.400786221479542}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons':

In [36]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


In [37]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of spiritual places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of spiritual places in Roma, RM =  125
Showing Top 100
Total number of spiritual places in Milano, MI =  84
Showing Top 100
Total number of spiritual places in Firenze, FI =  63
Showing Top 100


In [38]:
maps[cities[0]]

In [39]:
maps[cities[1]]

In [40]:
maps[cities[2]]

We can see that Rome seems to be the most dense cities with Spiritual places.

We will use some statistics to calculate the mean location of spiritual places which should be near to most of the points if they are really dense, otherwise they are far. Finally, we will calculate the average of the distance of the venues to the mean coordinates.

In [41]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Roma, RM
Mean Distance from Mean coordinates
0.02077963650484641
Milano, MI
Mean Distance from Mean coordinates
0.02751600469821392
Firenze, FI
Mean Distance from Mean coordinates
0.01428165435842807


In [42]:
maps[cities[0]]

In [43]:
maps[cities[1]]

In [44]:
maps[cities[2]]

We now see that Firenze seems to be the best option. 
However, Firenze is a lot smaller than Rome so it is less subjected to outliers. for example and in Rome we can see a few points being far away from the mean. So let's try removing it and calculate it again.

In [52]:
cities = ['Roma, RM', 'Firenze, FI']
for city in cities:
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

    print(city)
    print("Mean Distance from Mean coordinates")
    dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
    dists.sort()
    print(np.mean(dists[:-2]))# We are ignoring the 11 biggest distances



Roma, RM
Mean Distance from Mean coordinates
0.018969413571599394
Firenze, FI
Mean Distance from Mean coordinates
0.012461118006452401


## In this way, Roma takes the first place concerning density for spiritual places.